<a href="https://colab.research.google.com/github/johyunkang/MLwithPythonCookbook/blob/main/4_6_outlier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 4.6 이상치 감지하기

In [14]:
import numpy as np
from sklearn.covariance import EllipticEnvelope
from sklearn.datasets import make_blobs

# 모의 데이터 만들기
features, _ = make_blobs(n_samples=10,
                         n_features = 2,
                         centers = 1,
                         random_state = 1)

print('만들어진 샘플 데이터:\n')
print(features)

print('\n\n')
print('평균:', features.mean())

# 첫번째 샘플을 극단적인 값으로 변경
features[0, 0] = 10_000
features[0, 1] = 10_000

features[9, 0] = 10_000
features[9, 1] = 10_000


print('\n\n')
print('이상치 적용한 값\n')
print(features)

# 이상치 감지 객체 만들기
# contamination : 이상치가 있다고 생각하는 비율 e.g) .2 = 20%
# contamination 비율 보다 낮은 값을 이상치로 감지함
outlier_detector = EllipticEnvelope(contamination=.2) 
# 감지 객체 훈련
outlier_detector.fit(features)

# 이상치 예측
print('\n\n')
print('이상치가 있는 배열은 -1 리턴')
print(outlier_detector.predict(features))

만들어진 샘플 데이터:

[[-1.83198811  3.52863145]
 [-2.76017908  5.55121358]
 [-1.61734616  4.98930508]
 [-0.52579046  3.3065986 ]
 [ 0.08525186  3.64528297]
 [-0.79415228  2.10495117]
 [-1.34052081  4.15711949]
 [-1.98197711  4.02243551]
 [-2.18773166  3.33352125]
 [-0.19745197  2.34634916]]



평균: 1.1916761241172682



이상치 적용한 값

[[ 1.00000000e+04  1.00000000e+04]
 [-2.76017908e+00  5.55121358e+00]
 [-1.61734616e+00  4.98930508e+00]
 [-5.25790464e-01  3.30659860e+00]
 [ 8.52518583e-02  3.64528297e+00]
 [-7.94152277e-01  2.10495117e+00]
 [-1.34052081e+00  4.15711949e+00]
 [-1.98197711e+00  4.02243551e+00]
 [-2.18773166e+00  3.33352125e+00]
 [ 1.00000000e+04  1.00000000e+04]]



이상치가 있는 배열은 -1 리턴
[-1  1  1  1 -1  1  1  1  1 -1]


In [17]:
# IQR을 이용하여 이상치 구별
feature = features[:, 0]
print('feature:', feature)
print('feature type:', type(feature))

def indicies_of_outlier(x) :
    q1, q3 = np.percentile(x, [25, 75])
    iqr = q3 - q1
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)
    return np.where((x > upper_bound) | (x < lower_bound))

indicies_of_outlier(feature)

feature: [ 1.00000000e+04 -2.76017908e+00 -1.61734616e+00 -5.25790464e-01
  8.52518583e-02 -7.94152277e-01 -1.34052081e+00 -1.98197711e+00
 -2.18773166e+00  1.00000000e+04]
feature type: <class 'numpy.ndarray'>


(array([0, 9]),)

## 4.7 이상치 다루기

이상치를 다루는 방법은 크게 3가지임
- 이상치 삭제
- 이상치로 표시하고, 이를 특성의 하나로 포함
- 이상치의 영향이 줄어들도록 특성을 변환

In [26]:
import pandas as pd

houses = pd.DataFrame()
houses['price'] = [534_433, 392_333, 293_222, 4_322_032]
houses['bathrooms'] = [2, 3.5, 2, 116]
houses['square_feet'] = [1500, 2500, 1500, 48000]

display(houses)

# 이상치 다루는 법 1 : 샘플을 필터링
print('\n\n')
print('화장실이 20개 미만인  정보')
display(houses[houses['bathrooms'] < 20])


# 이상치 다루는 법 2 : 이상치를 표시
print('\n\n')
print('이상치를 표시')
houses['outlier'] = np.where(houses['bathrooms']<20, 'N', 'Y')
display(houses)


# 이상치 다루는 법 3 : 이상치 영향이 줄어들도록 특성을 변환
print('\n\n')
print('이상치 영향이 줄도록 특성 변환')
houses['log_of_square_feet'] = [np.log(x) for x in houses['square_feet']]
display(houses)


,price,bathrooms,square_feet
0,534433,2.0,1500
1,392333,3.5,2500
2,293222,2.0,1500
3,4322032,116.0,48000





화장실이 20개 미만인  정보


,price,bathrooms,square_feet
0,534433,2.0,1500
1,392333,3.5,2500
2,293222,2.0,1500





이상치를 표시


,price,bathrooms,square_feet,outlier
0,534433,2.0,1500,N
1,392333,3.5,2500,N
2,293222,2.0,1500,N
3,4322032,116.0,48000,Y





이상치 영향이 줄도록 특성 변환


,price,bathrooms,square_feet,outlier,log_of_square_feet
0,534433,2.0,1500,N,7.313220
1,392333,3.5,2500,N,7.824046
2,293222,2.0,1500,N,7.313220
3,4322032,116.0,48000,Y,10.778956


**이상치가 평균과 분산에 영향을 끼치기 때문에 이상치가 있다면 `표준화`가 적절하지 않음.**
** [RobustScaler](http://bit.ly/2DcgyNT)와 같이 이상치에 민감하지 않은 스케일링 방법을 사용하세요 **
